In [2]:
import pandas as pd
import numpy as np

#filter_v2는 감소한 주식이 없는걸 모두 제외함.

# 엑셀 파일 읽어오기
df = pd.read_excel("D:/stock/filtered_share_data/filter_v2.xlsx")

# 숫자 문자열에서 쉼표(,) 제거하고 정수로 변환하는 함수 정의
def convert_to_int(value):
    try:
        # 쉼표(,) 제거하고 정수로 변환
        return int(str(value).replace(',', ''))
    except ValueError:
        # 변환에 실패하면 그대로 반환
        return value

# 특정 열에 대해 convert_to_int 함수 적용
columns_to_convert = ['발행주식 총수', '현재까지 발행한 주식의 총수', '현재까지 감소한 주식의 총수', '이익소각', '감자', '발행주식의 총수', '자기주식수', '유통주식수']
for column in columns_to_convert:
    df[column] = df[column].apply(convert_to_int)

# 데이터프레임 확인
df.info()

# 열에서 특정 값 변환하기
target_value = '-'
replacement_value = 0  # 변환하고자 하는 값

# 열에서 특정 값(target_value)을 가진 행을 찾아 replacement_value로 변환
df['이익소각'] = df['이익소각'].apply(lambda x: replacement_value if x == target_value else x)
df['감자'] = df['감자'].apply(lambda x: replacement_value if x == target_value else x)
#df['발행주식'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6988 entries, 0 to 6987
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   회사명              6988 non-null   object
 1   발행주식 총수          6988 non-null   int64 
 2   현재까지 발행한 주식의 총수  6988 non-null   int64 
 3   현재까지 감소한 주식의 총수  6988 non-null   int64 
 4   감자               6988 non-null   object
 5   이익소각             6988 non-null   object
 6   발행주식의 총수         6988 non-null   object
 7   자기주식수            6988 non-null   object
 8   유통주식수            6988 non-null   object
 9   연도               6988 non-null   int64 
dtypes: int64(4), object(6)
memory usage: 546.1+ KB


In [45]:
#감소한 주식이 없거나 이익소각을 한 주식만 선택 
df2=df[       (df['현재까지 감소한 주식의 총수']!=0) & (df['이익소각'] > 0)        ]


# '현재까지 발행한 주식의 총수'가 증가하지 않거나 감소한 행을 필터링하는 함수 정의
def filter_non_increasing_or_decreasing(group):
    return (group['현재까지 발행한 주식의 총수'].equals(group['현재까지 발행한 주식의 총수'].shift()) or
            group['현재까지 발행한 주식의 총수'].is_monotonic_decreasing)

# '회사명'이 같은 그룹 내에서 '현재까지 발행한 주식의 총수'가 증가하지 않거나 감소한 행을 고름
result_df = df2.groupby('회사명').filter(filter_non_increasing_or_decreasing)

#위 케이스의 result_df 경우 삼성전자 처럼 액면분할 할 경우 골라내지 못함. 
result_df.to_excel("이익소각있고, 발행주식이 증가하지 않거나 감소한 케이스.xlsx", index=False, engine='openpyxl')
df2.to_excel("이익소각있고, 액면분할 포함된 케이스.xlsx", index=False, engine='openpyxl')

In [10]:
df_use= pd.read_excel('이익소각있고, 액면분할 포함된 케이스.xlsx')

# '이익소각' 열이 증가한 행만 필터링
filtered_df = df.loc[df.groupby('회사명')['이익소각'].diff().fillna(0) > 0]

filtered_df.to_excel("이익소각이 증가한 케이스가 존재하는 회사만 필터링.xlsx", index=False, engine='openpyxl')


In [18]:
len(filtered_df['회사명'].unique())

87

In [17]:
len(df['회사명'].unique())

1204

In [19]:
origin = pd.read_excel("D:/stock/filtered_share_data/share_total.xlsx")
len(origin['회사명'].unique())

2639

In [20]:
filtered_df['회사명'].unique()

array(['휴젤', 'ES큐브', '화인베스틸', '오성첨단소재', '리드코프', '랩지노믹스', '그래디언트', '에스엠화진',
       'APS', '콜마파마', 'JYP Ent.', '나눔테크', '한일홀딩스', '유비쿼스홀딩스', '대아티아이',
       '네오위즈홀딩스', '덕신하우징', '동양고속', '동아엘텍', 'SK하이닉스', '한솔홈데코', '메리츠금융지주',
       'SK텔레콤', '한미반도체', '테크윙', 'KB금융', '메리츠화재해상보험', '제노코', '토비스', '펄어비스',
       '세진중공업', '한국정보통신', '기업은행', '아나패스', '씨엔플러스', '현대자동차', '금호석유화학',
       '탑선', '유티아이', '성창기업지주', '실리콘투', '호전실업', '백산', '현대모비스', '네패스',
       '동남합성', '비나텍', '코웨이', '삼영엠텍', '폴라리스우노', '와이지-원', '두산', '상상인',
       '한화에어로스페이스', 'NAVER', '하림지주', '금호에이치티', '삼성전자', '휠라홀딩스', '메리츠증권',
       '메디쎄이', '미원홀딩스', '미원화학', '톱텍', 'KG스틸', 'TS인베스트먼트', 'KSS해운', '엠씨넥스',
       'HD현대', '아이윈플러스', 'JW중외제약', '크레버스', '한화솔루션', '엔브이에이치코리아', '락앤락',
       'MDS테크', '카나리아바이오', 'CJ ENM', '세종텔레콤', '미원상사', 'HL홀딩스', 'HL D&I',
       '신한지주', '지엔씨에너지', 'POSCO홀딩스', '한중엔시에스', '아이마켓코리아'], dtype=object)